In [71]:
from sklearn import preprocessing
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping

1.	What is a neural network? What are the general steps required to build a neural network? 

Neural networks are computing systems that are inspired by the neural networks in brains. They learn by example and can classify and make predictions.  

What are the steps? 

Clean the data

Select the architecture

train the neural network

tweak the performance

test

deploy model


2.	Generally, how do you check the performance of a neural network? Why? 


There are a bunch of methods to check the performance 

# regression

MSE

NMSE

RMSE

R Squared

# For classification

TP rate

FP Rate

F Measure

Acurracy

Precision

Recall

Mean squared error is most common for regression,

Why do we want to check the performance? Because having an accurate model is the only reason for making a model. It is pointless to have a model with terrible performance. If you check the performance you can tweak a ton of different options to try to make it perform better. 

3.	Create a neural network using keras to predict the outcome of either of these datasets: 
Cardiac Arrhythmia: https://archive.ics.uci.edu/ml/datasets/Arrhythmia 
Abalone age: https://archive.ics.uci.edu/ml/datasets/Abalone

We preprocessed this last week so I saved the final version to use here. 


In [72]:
abalone_df  = pd.read_csv(r"C:\Users\weird\Documents\Data_Science_CG_2021\homework\week_18.csv")
abalone_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


In [73]:
X = abalone_df.drop(['Rings'], axis=1)
y = abalone_df['Rings']


In [74]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27)

In [75]:
sc= StandardScaler()
X=sc.fit_transform(X)


In [95]:

n_cols = X.shape[1]
model = Sequential()
model.add(Dense(300, activation='relu', input_shape = (n_cols,)))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='relu'))
model.summary()

early_stopping_monitor = EarlyStopping(patience = 3)



model.summary()
# Compile the model
#model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mean_squared_error'])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','RootMeanSquaredError'])

#model.fit(X_train, y_train,  validation_split = .3, epochs = 30, batch_size=100,verbose=1,callbacks = [early_stopping_monitor])
model.fit(X, y, validation_split = .3, epochs = 30, batch_size=100,verbose=1,callbacks = [early_stopping_monitor])                         

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 300)               3300      
_________________________________________________________________
dense_46 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_47 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_48 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 301       
Total params: 274,501
Trainable params: 274,501
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_9"
_________________________________________________________________
Layer (type)              

In [96]:

NNRMSE = 1.8705
NNRMSE

1.8705

In [78]:
abalone_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


4.	Write another algorithm to predict the same result as the previous question using either KNN or logistic regression.




In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27)

In [98]:
from sklearn.metrics import mean_squared_error 

LR = LinearRegression()

LR.fit(X_train, y_train)

y_pred = LR.predict(X_test)


mse = mean_squared_error(y_test, y_pred)

rmse = mse**.5
rmse

1.9119603708535091

In [100]:
LogReg = LogisticRegression(max_iter = 7500, random_state = 27)
LogReg.fit(X_train, y_train)
LogRegy_pred = LogReg.predict(X_test)

MSE = mean_squared_error(y_test, LogRegy_pred)

RMSE = MSE**.5
RMSE

2.1654728128246297

5.	Create a neural network using pytorch to predict the same result as question 3. 


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = abalone_df.drop('Rings', axis=1).values
y = abalone_df['Rings'].values

# Split into training and test set
PTX_train, PTX_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=27)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(PTX_train)
X_test=sc.fit_transform(PTX_test)

In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

#based on error messages further down, I think these should be float, but it got 
#very angry at me when I tried to change them to FloatTensor so we'll just 
#cast to float later. 
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8600, -0.9299, -0.6180,  ..., -0.6738,  1.4274, -0.7476],
        [ 0.5688,  0.6483,  0.3110,  ..., -0.6738, -0.7006,  1.3377],
        [ 1.5774,  1.6156,  1.7709,  ..., -0.6738, -0.7006,  1.3377],
        ...,
        [-0.1036, -0.0135, -0.3525,  ..., -0.6738, -0.7006,  1.3377],
        [-0.9440, -0.7772, -0.8834,  ..., -0.6738,  1.4274, -0.7476],
        [-0.0195, -0.2680, -0.3525,  ..., -0.6738,  1.4274, -0.7476]])


In [85]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=10, hidden1=20, hidden2=20, out_features =1):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [86]:
torch.manual_seed(27)

#instantiate the model
model = ANN_Model()

In [87]:
# loss function
loss_function = nn.MSELoss()


In [88]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


In [92]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 250
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred.float(), y_train.float())
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 7.701865196228027
Epoch number: 11 with loss: 7.699443817138672
Epoch number: 21 with loss: 7.6972455978393555
Epoch number: 31 with loss: 7.695244789123535
Epoch number: 41 with loss: 7.693408012390137
Epoch number: 51 with loss: 7.691705703735352
Epoch number: 61 with loss: 7.6901421546936035
Epoch number: 71 with loss: 7.688711643218994
Epoch number: 81 with loss: 7.687398433685303
Epoch number: 91 with loss: 7.686204433441162
Epoch number: 101 with loss: 7.685123920440674
Epoch number: 111 with loss: 7.6841349601745605
Epoch number: 121 with loss: 7.6832275390625
Epoch number: 131 with loss: 7.682391166687012
Epoch number: 141 with loss: 7.681618690490723
Epoch number: 151 with loss: 7.680904388427734
Epoch number: 161 with loss: 7.68024206161499
Epoch number: 171 with loss: 7.679628372192383
Epoch number: 181 with loss: 7.679061412811279
Epoch number: 191 with loss: 7.678535461425781
Epoch number: 201 with loss: 7.678049087524414
Epoch number: 211 with l

In [93]:
torchRMSE = 7.676455497741699**.5
torchRMSE 



2.7706417122648137

6.	Compare the performance of the neural networks to the other model you created. Which performed better? Why do you think that is?


My Keras model performed the best. My guess is that by design it is better than linear or logistic regression. I think if I had a more solid understanding of what was happening in the pytorch version I might be able to get that down and better.

In [101]:
print('Keras: ' + str(NNRMSE))
print('LogisticRegression: ' + str(RMSE))
print('LinearRegression: ' + str(rmse))
print('Pytorch: ' + str(torchRMSE))



Keras: 1.8705
LogisticRegression: 2.1654728128246297
LinearRegression: 1.9119603708535091
Pytorch: 2.7706417122648137
